In [ ]:
img = '/mnt/d/autocls01/dadosfrancisco/img_sample.tif'

rowspx, colspx = 1000, 1000

out = '/mnt/d/autocls01/dadosfrancisco/'

bname = 'terceira'

In [ ]:
import os
from osgeo import gdal, ogr, osr
import numpy as np
from shapely.geometry import Polygon
import geopandas as gp

In [ ]:
src = gdal.Open(img)

In [ ]:
proj = osr.SpatialReference(wkt=src.GetProjection())
    
epsg = int(str(proj.GetAttrValue('AUTHORITY', 1)))

In [ ]:
epsg

In [ ]:
left, cellx, _, top, __, celly = src.GetGeoTransform()

In [ ]:
ncols = src.RasterXSize
nrows = src.RasterYSize

In [ ]:
right = left + (ncols * cellx)
bottom = top + (nrows * celly)

In [ ]:
npartescols = ncols / colspx
npartescols = int(npartescols) if npartescols == int(npartescols) else int(npartescols) + 1

npartesrows = nrows / rowspx
npartesrows = int(npartesrows) if npartesrows == int(npartesrows) else int(npartesrows) + 1

In [ ]:
min_x = left
max_y = top
for r in range(npartesrows):
    min_y = max_y + (celly * rowspx)

    if min_y < bottom:
        min_y = bottom

    for c in range(npartescols):
        max_x = min_x + (cellx * colspx)

        if max_x > right:
            max_x = right

        # Create bounding box polygon
        points = [
            # Top left
            (min_x, max_y),
            # Top Right
            (max_x, max_y),
            # Bootom Right
            (max_x, min_y),
            # Bottom left
            (min_x, min_y),
            # Top Left
            (min_x, max_y)
        ]

        polygon = Polygon(points)

        gdf = gp.GeoDataFrame([[1]], columns=['pid'], crs=f"EPSG:{str(epsg)}", geometry=[polygon])

        gdf.to_file(os.path.join(out, f'{bname}_{str(r)}_{str(c)}.shp'))

        # Create Raster
        _rows = (max_y - min_y) / abs(celly)
        _cols = (max_x - min_x) / cellx

        _rows = int(_rows) if _rows == int(_rows) else int(_rows) + 1
        _cols = int(_cols) if _cols == int(_cols) else int(_cols) + 1

        rstval = np.full((_rows, _cols), 1)

        rstpath = os.path.join(out, f'{bname}_{str(r)}_{str(c)}.tif')

        nimg = gdal.GetDriverByName('GTiff').Create(
            rstpath, _cols, _rows, 1, gdal.GDT_Byte
        )

        nimg.SetGeoTransform((min_x, cellx, 0, max_y, 0, celly))

        band = nimg.GetRasterBand(1)
    
        band.WriteArray(rstval)
        
        rstSrs = osr.SpatialReference()
        rstSrs.ImportFromEPSG(epsg)
        nimg.SetProjection(rstSrs.ExportToWkt())
    
        band.FlushCache()

        min_x = min_x + (cellx * colspx)

    max_y = max_y + (celly * rowspx)
    min_x = left